# Example of next_pass application to Hurricane Milton, Florida, October 2024

## Background
* Knowing the time of a satellite overpass (OP) at a precise location is crucial to plan and prepare disaster impact studies. 
The script below can be used to predict the overpasses of the Landsat 8 & 9 and Sentinel 1 & 2 satellites over a  selected location. For Landsat 8 this occurs every 16 days and for Sentinel 2A / 2B this occurs every 10 days.

* The code calls the Python package 'next_pass' located at https://github.com/ehavazli/next_pass. The latter predicts the next overpass of the satellite of interest by scanning the relevant acquisition plans:
	- Landsat acquisition plans (json files) : https://landsat.usgs.gov/sites/default/files/landsat_acq/assets/json/cycles_full.json
	- Sentinel acquisition plans (KML files to import to Google Earth Pro) : https://sentinel.esa.int/web/sentinel/copernicus/sentinel-1/acquisition-plans
      

* This specific notebook showcases the application of the next_pass tool to hurricane Milton in Florida between October 3rd and October 11th, 2024 (https://maps.disasters.nasa.gov/arcgis/apps/MinimalGallery/index.html?appid=081982abfd5f4732b45df6db9f6de35e). The user can set the location using the "MiltonHurricaneFlorida" kml file 

## Tool Description

All what a user needs to provide is the precise location for which he desires to identify the next overpasses. The location can be input as a single point(latitude, longitude), or bounding box (SNEW coordinates) or a location file path (.kml). The script returns the next collect for Sentinel-1 and Sentinel-2 and the next passes, in ascending and descending directions separately, for Landsat-8 and Landsat-9.

- Specify a location 
- Run find_next_overpass for Sentinel-1, Sentinel-2 and the Landsats (8&9) 
- Visualize each of the above predicted overpass 

The outputs of next_pass can be compared against overpasses of the site you are interested in using the ESA Orbital Prediction and Overpass Tool (OPOT) at https://evdc.esa.int/orbit/ 


## Getting started
To run the overpass predictor with the given location, run all cells in the notebook starting with the "Load packages" cell.

### Load packages and functions

In [1]:
import sys
import os

# Add the parent directory to Python's path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    
import next_pass
import utils
import folium
import re  # Import regular expressions
import random  # To generate random colors
import earthaccess
import leafmap
import pandas as pd
import geopandas as gpd
import boto3
import rasterio
import rasterio as rio
import xarray as xr
from shapely.geometry import Point, Polygon, shape, box
from geopy.geocoders import Nominatim
from argparse import Namespace
from datetime import datetime
from rasterio.session import AWSSession
from dateutil.relativedelta import relativedelta

In [2]:
import colorsys

# Style function for the bounding box GeoJSON layer
def style_function(feature):
    return {
        'fillColor': '#808080',  # Gray fill color
        'color': '#000000',       # Black border color
        'weight': 4,              # Thicker border (increased thickness)
        'fillOpacity': 0.3        # Fill opacity (adjust if needed)
    }
# Function to generate random hex color
def random_color():
    return "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Function to print text with color in console (ANSI escape code)
def print_colored_text(text, color):
    # Escape sequence for colored text
    print(f"\033[38;2;{color[0]};{color[1]};{color[2]}m{text}\033[39m")

# 
def hsl_distinct_colors(n):
    colors = []
    for i in range(n):
        # Generate colors with different hues
        hue = i / float(n)  # Hue ranges from 0 to 1
        color = colorsys.hsv_to_rgb(hue, 1.0, 1.0)  # Convert HSL to RGB
        # Convert from RGB (0-1) to hex (#RRGGBB)
        rgb = [int(c * 255) for c in color]
        hex_color = "#{:02x}{:02x}{:02x}".format(*rgb)
        colors.append(hex_color)
    return colors

def spread_rgb_colors(n):
    colors = []
    step = 255 // n  # Divide the color space into n parts
    for i in range(n):
        # Spread out the color values across the RGB spectrum
        r = (i * step) % 256
        g = ((i + 1) * step) % 256
        b = ((i + 2) * step) % 256
        hex_color = "#{:02x}{:02x}{:02x}".format(r, g, b)
        colors.append(hex_color)
    return colors

def hsl_distinct_colors_improved(num_colors):
    colors = []
    
    for i in range(num_colors):
        # Set Hue (H) to a random value, excluding extremes like 0° (red) and 60° (yellow)
        hue = (i * 360 / num_colors) % 360
        
        # Set Saturation (S) to a high value (e.g., 70%) for vivid colors
        saturation = random.randint(60, 80)  # Avoid dull colors
        
        # Set Lightness (L) to a lower value to avoid bright, light colors like yellow (range 30-50%)
        lightness = random.randint(30, 50)  # Darker or neutral colors

        # Convert HSL to RGB using the colorsys library
        r, g, b = colorsys.hls_to_rgb(hue / 360, lightness / 100, saturation / 100)

        # Convert RGB to hex format (RGB values are in [0, 1], so multiply by 255)
        hex_color = "#{:02x}{:02x}{:02x}".format(int(r * 255), int(g * 255), int(b * 255))
        colors.append(hex_color)
    
    return colors

# Specify location
Start with selecting the location by  specifying the latitude/longitude (single point or bounding box) or pointing the script to the path of a location file (.kml). Please set location_file_path to '' if using the single point or boundng box options. 

In [3]:
# Floods near Porto Alegre, Brazil in 2024
lon_W, lat_S, lon_E, lat_N = (-54.215, -30.766,-50.814, -28.938)
location_file_path ='locations/MiltonHurricaneFlorida.kml' # Hurricane Milton location for this example

### Specify satellites of interest 
For now, the tool operates for Sentinel 1A and 2A and Landsat 8 and 9.

In [4]:
# Satellites
sat1 = "sentinel-1"
sat2 = "sentinel-2"
sat3 = "landsat"

### Run next_pass
use next_pass to predict the overpasses of the above satellites over the selected location. The cells below will call next_pass to predict overpasses for the selected satellites, then provide overpasses vizualisation in an interactive map.

In [5]:
# Create the args object to pass to find_next_overpass
args = Namespace(
    bbox=(lat_S, lat_N, lon_W, lon_E),
    aoi_file=location_file_path,  # assuming locationfile_path is defined
    satellite=sat1  # assuming sat1 is defined (e.g., 'sentinel-1', 'sentinel-2', 'landsat')
)

In [6]:
print("*** ",sat1," ***")
result1 = next_pass.find_next_overpass(args)
# result1 is a dictionary 
s1_next_collect_info = result1.get("next_collect_info", "No collection info available")
s1_next_collect_geometry = result1.get("next_collect_geometry", None)
print(s1_next_collect_info)

***  sentinel-1  ***
+-----+--------------------------+------------------+
|   # | Collection Date & Time   |   Relative Orbit |
+=====+==========================+==================+
|   1 | 2025-04-25 23:23:15      |               48 |
+-----+--------------------------+------------------+
|   2 | 2025-04-30 23:31:35      |              121 |
+-----+--------------------------+------------------+
|   3 | 2025-05-02 23:15:08      |              150 |
+-----+--------------------------+------------------+


In [7]:
print("*** ",sat2," ***")
args.satellite = sat2
result2 = next_pass.find_next_overpass(args)
s2_next_collect_info = result2.get("next_collect_info", "No collection info available")
s2_next_collect_geometry = result2.get("next_collect_geometry", None)
print(s2_next_collect_info)

***  sentinel-2  ***
+-----+--------------------------+------------------+
|   # | Collection Date & Time   |   Relative Orbit |
+=====+==========================+==================+
|   1 | 2025-04-22 15:58:13      |               97 |
+-----+--------------------------+------------------+


In [8]:
print("*** ",sat3," ***")
args.satellite = sat3
result3 = next_pass.find_next_overpass(args)

***  landsat  ***
+-------------+--------+-------+-----------+-------------------------------------------------------+
| Direction   |   Path |   Row | Mission   | Next Passes                                           |
+=============+========+=======+===========+=======================================================+
| Ascending   |    107 |   163 | Landsat_8 | 4/27/2025, 5/13/2025, 5/29/2025, 6/14/2025, 6/30/2025 |
+-------------+--------+-------+-----------+-------------------------------------------------------+
| Ascending   |    107 |   163 | Landsat_9 | 5/5/2025, 5/21/2025, 6/6/2025, 6/22/2025, 7/8/2025    |
+-------------+--------+-------+-----------+-------------------------------------------------------+
| Descending  |    223 |    81 | Landsat_8 | 4/23/2025, 5/9/2025, 5/25/2025, 6/10/2025, 6/26/2025  |
+-------------+--------+-------+-----------+-------------------------------------------------------+
| Descending  |    223 |    81 | Landsat_9 | 5/1/2025, 5/17/2025, 6/2/202

### Overpasses Vizualisation  
The below vizualization tool shows the path of a selected satellite at the predicted date/time

In [9]:
# Start by choosing what satellite to visualize 
sat_to_visualize = 'Sentinel-1'; # can be Sentinel-1 or Sentinel-2

if location_file_path:
    area_polygon = utils.create_polygon_from_kml(location_file_path)
    # Convert to a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': [area_polygon]}, crs="EPSG:4326")  # WGS 84 CRS
    # Create a Folium map centered at the bounding box centroid
    m = folium.Map(location=[area_polygon.centroid.y, area_polygon.centroid.x], zoom_start=4)
    # Add the bounding box as a GeoJSON layer
    folium.GeoJson(gdf.to_json(), name="Area of interest", style_function=style_function).add_to(m)
elif lat_S == lat_N and lon_W == lon_E:
    # Create the point
    point = Point(lon_E, lat_N)

    # Create a Folium map centered at the point location
    m = folium.Map(location=[point.y, point.x], zoom_start=4)

    # Add a cross-shaped marker to the map
    folium.Marker(
        location=[point.y, point.x],  # Latitude, Longitude
        icon=folium.Icon(icon='glyphicon-remove', icon_color='red', prefix='glyphicon')  # Cross symbol with red color
    ).add_to(m)
else:
    # Create the bounding box as a polygon
    bounding_box = box(lon_W, lat_S, lon_E, lat_N)

    # Convert to a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': [bounding_box]}, crs="EPSG:4326")  # WGS 84 CRS

    # Create a Folium map centered at the bounding box centroid
    m = folium.Map(location=[bounding_box.centroid.y, bounding_box.centroid.x], zoom_start=4)
    # Add the bounding box as a GeoJSON layer
    folium.GeoJson(gdf.to_json(), name="Bounding Box", style_function=style_function).add_to(m)

if sat_to_visualize == 'Sentinel-1':
    vi_next_collect_info = s1_next_collect_info
    vi_next_collect_geometry = s1_next_collect_geometry
elif sat_to_visualize == 'Sentinel-2':
    vi_next_collect_info = s2_next_collect_info
    vi_next_collect_geometry = s2_next_collect_geometry
else:
    vi_next_collect_info = l8_next_collect_info
    vi_next_collect_geometry = l8_next_collect_geometry
        
print('\n ** Visualizing overpasses for ',sat_to_visualize,' ** \n')
# Add each Polygon in next_collect_geometry
lines = vi_next_collect_info.split("\n")
# Clean lines by keeping only those that contain numbers (1-9)
cleaned_info = [line for line in lines if re.search(r'[1-9]', line)]  # Line must contain digits (1-9)
vi_next_collect_info_list = cleaned_info  # Now it's a list of strings (one per row in the table)
num_polygons = len(vi_next_collect_geometry)
num_info_lines = len(vi_next_collect_info_list)
#print(num_polygons)
#print(num_info_lines)

# Use the HSL distinct colors function
distinct_colors_list_1 = spread_rgb_colors(num_polygons)
distinct_colors_list_2 = hsl_distinct_colors(num_polygons)
distinct_colors_list_3 = hsl_distinct_colors_improved(num_polygons)

if vi_next_collect_geometry:
    for i, (polygon, info) in enumerate(zip(vi_next_collect_geometry, vi_next_collect_info_list), start=1):
    #i=3 
    #polygon =vi_next_collect_geometry[2]
    #info = vi_next_collect_info_list[2]
    
        if isinstance(polygon, Polygon):  # Ensure it's a valid Polygon
            # Get a distinct color for each polygon
            color = distinct_colors_list_3[i - 1]
    
            # Print the info with corresponding color in the console
            print_colored_text(f"{info}", tuple(int(color[i:i+2], 16) for i in (1, 3, 5)))
    
            
            geojson_data = gpd.GeoSeries([polygon]).__geo_interface__
            folium.GeoJson(
                geojson_data, 
                name="Next Collect Area",
                style_function=lambda x, color=color: {"color": color, "weight": 2, "fillOpacity": 0.3},
                popup=folium.Popup(f"Polygon: {info}", max_width=300)  # Display corresponding info line
            ).add_to(m)

print('')
# Display the map and save to file
m.save(sat_to_visualize+"_Next_Overpasses.html")
m  # If using Jupyter Notebook


 ** Visualizing overpasses for  Sentinel-1  ** 

|   1 | 2025-04-25 23:23:15      |               48 |
|   2 | 2025-04-30 23:31:35      |              121 |
|   3 | 2025-05-02 23:15:08      |              150 |



# Available OPERA Products

## Description
* Here we will use Leafmap and Earthaccess to explore OPERA DSWx Products.
* The Leafmap library provides a suite of tools for interactive mapping and visualization in Jupyter Notebooks. Leafmap version 0.30.0 and later offers tools specifically for accessing NASA Earthdata by building on the newly developed NASA Earthaccess library. Earthaccess provides streamlined access to NASA Earthdata and simplifies the authentication and querying process over previously developed approaches. 
* This section is designed to leverage tools within Earthaccess and Leafmap to facilitate easier access and visualization of OPERA data products for a user-specified area of interest (AOI). 

## OPERA DSWx Products 

The Dynamic Surface Water eXtent (DSWx) products map pixel-wise surface water detections using optical or SAR imagery. The DSWx suite is composed of complementary products, which are named according to their input datasets. In the present section, we will focus on: 

- DSWx from Harmonized Landsat Sentinel-2 (DSWx-HLS)
- DSWx from Sentinel-1 (DSWx-S1)


## View the available OPERA products
Note above that the `earthdata_df` contains a number of columns with metadata about each available product. the `ShortName` column will be used to produce a new dataframe containing only OPERA products. Let's view the available products and their metadata.

In [10]:
### View Earthdata datasets
earthdata_url = 'https://github.com/opengeos/NASA-Earth-Data/raw/main/nasa_earth_data.tsv'
earthdata_df = pd.read_csv(earthdata_url, sep='\t')
opera_df = earthdata_df[earthdata_df['ShortName'].str.contains('OPERA', case=False)]
### Print the available OPERA datasets 
print('Available OPERA datasets:', opera_df['ShortName'].values)

Available OPERA datasets: ['OPERA_DSWX-S1_CALVAL_V1' 'OPERA_L2_CSLC-S1-STATIC_V1'
 'OPERA_L2_CSLC-S1_V1' 'OPERA_L2_RTC-S1-STATIC_V1' 'OPERA_L2_RTC-S1_V1'
 'OPERA_L3_DISP-S1_V1' 'OPERA_L3_DIST-ALERT-HLS_V1'
 'OPERA_L3_DIST-ANN-HLS_V1' 'OPERA_L3_DSWX-HLS_V1' 'OPERA_L3_DSWX-S1_V1']


## Display most recent OPERA products at the selected Area of Interest (AOI)
* The script will use the area of interest indicated above (to predict overpasses) to retreive the five most recent OPERA products at the time the present notebook is run.

In [11]:
### This cell initializes the AOI.
if location_file_path:
    AOI = area_polygon.bounds
else:
    # Create the bounding box as a polygon
    AOI = (lon_W, lat_S,lon_E, lat_N)
print(AOI)
c_lat = (AOI[1] + AOI[3]) / 2  # (miny + maxy) / 2
c_lon = (AOI[0] + AOI[2]) / 2  # (minx + maxx) / 2

(-82.35238247646188, 27.9751200513834, -80.56469917569552, 28.84709329637735)


In [12]:
# define pre and Syn event dates in the last month
from datetime import date
today = date.today()
one_year_ago = date.today() - relativedelta(months=12)
StartDate_Recent = one_year_ago.strftime("%Y-%m-%d") + "T00:00:00"  
EndDate_Recent = today.strftime("%Y-%m-%d") + "T23:59:59" 
print(StartDate_Recent)
print(EndDate_Recent)

2024-04-21T00:00:00
2025-04-21T23:59:59


In [13]:
opera_datasets = opera_df['ShortName'].values
# Dictionary to store results
results_dict = {}

# Iterate through each dataset
for dataset in opera_datasets:
    print(f"🔍 Searching {dataset}...")

    try:
        results, gdf = leafmap.nasa_data_search(
            short_name=dataset,
            cloud_hosted=True,
            bounding_box=AOI,
            temporal=(StartDate_Recent, EndDate_Recent),
            #count=-1,
            return_gdf=True,
        )
        gdf=gdf[-5:]
        print(f"✅ Success: {dataset} → {len(gdf)} granules found.")
        results_dict[dataset] = {
            "results": results,
            "gdf": gdf,
        }
    except Exception as e:
        print(f"❌ Error fetching {dataset}")


🔍 Searching OPERA_DSWX-S1_CALVAL_V1...
❌ Error fetching OPERA_DSWX-S1_CALVAL_V1
🔍 Searching OPERA_L2_CSLC-S1-STATIC_V1...
❌ Error fetching OPERA_L2_CSLC-S1-STATIC_V1
🔍 Searching OPERA_L2_CSLC-S1_V1...
✅ Success: OPERA_L2_CSLC-S1_V1 → 5 granules found.
🔍 Searching OPERA_L2_RTC-S1-STATIC_V1...
❌ Error fetching OPERA_L2_RTC-S1-STATIC_V1
🔍 Searching OPERA_L2_RTC-S1_V1...
✅ Success: OPERA_L2_RTC-S1_V1 → 5 granules found.
🔍 Searching OPERA_L3_DISP-S1_V1...
❌ Error fetching OPERA_L3_DISP-S1_V1
🔍 Searching OPERA_L3_DIST-ALERT-HLS_V1...
✅ Success: OPERA_L3_DIST-ALERT-HLS_V1 → 5 granules found.
🔍 Searching OPERA_L3_DIST-ANN-HLS_V1...
❌ Error fetching OPERA_L3_DIST-ANN-HLS_V1
🔍 Searching OPERA_L3_DSWX-HLS_V1...
✅ Success: OPERA_L3_DSWX-HLS_V1 → 5 granules found.
🔍 Searching OPERA_L3_DSWX-S1_V1...
✅ Success: OPERA_L3_DSWX-S1_V1 → 5 granules found.


In [14]:
m = leafmap.Map(center=(c_lat, c_lon), zoom=7)
m.add_basemap("Satellite")

num_items = len(results_dict.items())
distinct_colors_list_items = hsl_distinct_colors_improved(num_items)

for i, (dataset, data) in enumerate(results_dict.items()):
    color = distinct_colors_list_items[i - 1]
    if not data["gdf"].empty:
        print_colored_text(f"{dataset}", tuple(int(color[i:i+2], 16) for i in (1, 3, 5)))
        style = {"color": color, "fillColor": color, "weight": 2, "fillOpacity": 0.5}
        m.add_gdf(data["gdf"], layer_name=dataset, style=style)

m

OPERA_L2_CSLC-S1_V1
OPERA_L2_RTC-S1_V1
OPERA_L3_DIST-ALERT-HLS_V1
OPERA_L3_DSWX-HLS_V1
OPERA_L3_DSWX-S1_V1


Map(center=[28.411106673880376, -81.4585408260787], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Display available OPERA products at the selected Area of Interest (AOI) and a Time of interest (TOI)
* Now, we retrieve OPERA products available for this AOI at a specific time interval of interest (TOI) defined by the user related to a specific previous event that the user would like to consider.  

### Specify time interval 
The user can set a period for the retrieval of OPERA products if interested in a specific prior event. 
The user can also precise the OPERA products of interest from the list above, the cells below will display all requested OPERA products at the selected period of interest.  

In [15]:
# Here we define a time period for the retrieval of OPERA products of interest for the specific event
PreEventDate="2024-10-03"  #Pre-event date
SynEventDate="2024-10-11"  #Syn-event date

#### Query the OPERA DSWx-HLS dataset for the AOI

In [16]:
# Based on the period the user selected above, we define the four dates/times below. 
StartDate_PreEvent = PreEventDate + "T00:00:00"  #Pre-event image start date
EndDate_PreEvent = PreEventDate + "T23:59:59"    #Pre-event image end date

StartDate_SynEvent = SynEventDate + "T00:00:00"  #Syn-event image start date
EndDate_SynEvent= SynEventDate + "T23:59:59"    #Syn-event image end date

try:
    dswx_results_PreEvent, dswx_gdf_PreEvent = leafmap.nasa_data_search(
        short_name='OPERA_L3_DSWX-HLS_V1',
        cloud_hosted=True,
        bounding_box= AOI,
        temporal=(StartDate_PreEvent, EndDate_PreEvent),
        count=-1,  # use -1 to return all datasets
        return_gdf=True,
    )
    dswx_results_SynEvent, dswx_gdf_SynEvent = leafmap.nasa_data_search(
        short_name='OPERA_L3_DSWX-HLS_V1',
        cloud_hosted=True,
        bounding_box= AOI,
        temporal=(StartDate_SynEvent, EndDate_SynEvent),
        count=-1,  # use -1 to return all datasets
        return_gdf=True,
    )
except Exception as e:
    print(f"❌ Error fetching {'OPERA_L3_DSWX-HLS_V1'}")

In [17]:
dswx_results_PreEvent[0] #Note this just shows a single MGRS/HLS tile


Collection: {'Version': '1.0', 'ShortName': 'OPERA_L3_DSWX-HLS_V1'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -81.05029455895425, 'SouthBoundingCoordinate': 27.976885922012325, 'EastBoundingCoordinate': -79.92336418128741, 'NorthBoundingCoordinate': 28.972139864314535}]}}}
Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2024-10-03T15:55:16.990Z', 'BeginningDateTime': '2024-10-03T15:55:16.990Z'}}
Size(MB): 24.74973964691162
Data: ['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B01_WTR.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B02_BWTR.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B03_CONF.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B04_DIAG.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B05_WTR-1.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B06_WTR-2.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B07_LAND.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B08_SHAD.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B09_CLOUD.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_20241005T024554Z_L8_30_v1.0_B10_DEM.tif']

In [18]:
dswx_results_SynEvent[0] #Note this just shows a single MGRS/HLS tile

Collection: {'Version': '1.0', 'ShortName': 'OPERA_L3_DSWX-HLS_V1'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -81.05029455895425, 'SouthBoundingCoordinate': 27.976885922012325, 'EastBoundingCoordinate': -79.92336418128741, 'NorthBoundingCoordinate': 28.972139864314535}]}}}
Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2024-10-11T16:05:27.713Z', 'BeginningDateTime': '2024-10-11T16:05:27.713Z'}}
Size(MB): 27.073701858520508
Data: ['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B01_WTR.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B02_BWTR.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B03_CONF.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B04_DIAG.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B05_WTR-1.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B06_WTR-2.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B07_LAND.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B08_SHAD.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B09_CLOUD.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_V1/OPERA_L3_DSWx-HLS_T17RNM_20241011T160509Z_20241013T030340Z_S2B_30_v1.0_B10_DEM.tif']

### View the DSWx-HLS metadata and footprints

In [19]:
dswx_gdf_PreEvent.head()

,size,concept-type,concept-id,revision-id,native-id,collection-concept-id,provider-id,format,revision-date,EndingDateTime,...,RelatedUrls,InputGranules,ArchiveAndDistributionInformation,DayNightFlag,Identifiers,ProductionDateTime,URL,Name,bbox,geometry
0,24.749740,granule,G3257775271-POCLOUD,1,OPERA_L3_DSWx-HLS_T17RNM_20241003T155516Z_2024...,C2617126679-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-05T03:11:14.849Z,2024-10-03T15:55:16.990Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,"[/home/conda/input_dir/GSHHS_f_L1.dbf, /home/c...","[{'SizeUnit': 'MB', 'Size': 3.0517578125e-05, ...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-HLS_T17RNM_2024...,2024-10-05T02:45:54.003Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-81.05029455895425, 27.976885922012325, -79.9...","POLYGON ((-79.92336 27.97689, -79.92336 28.972..."
1,38.592364,granule,G3257775644-POCLOUD,1,OPERA_L3_DSWx-HLS_T17RLM_20241003T155516Z_2024...,C2617126679-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-05T03:12:31.256Z,2024-10-03T15:55:16.990Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,"[/home/conda/input_dir/GSHHS_f_L1.dbf, /home/c...","[{'SizeUnit': 'MB', 'Size': 3.0517578125e-05, ...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-HLS_T17RLM_2024...,2024-10-05T02:46:14.285Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-83.10254936908628, 27.965249423558927, -81.9...","POLYGON ((-81.967 27.96525, -81.967 28.96861, ..."
2,52.188193,granule,G3257777209-POCLOUD,1,OPERA_L3_DSWx-HLS_T17RMM_20241003T155516Z_2024...,C2617126679-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-05T03:13:36.835Z,2024-10-03T15:55:16.990Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,"[/home/conda/input_dir/GSHHS_f_L1.dbf, /home/c...","[{'SizeUnit': 'MB', 'Size': 3.0517578125e-05, ...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-HLS_T17RMM_2024...,2024-10-05T02:46:16.811Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-82.07663581871259, 27.976885922012325, -80.9...","POLYGON ((-80.94971 27.97689, -80.94971 28.972..."
3,31.442208,granule,G3257777608-POCLOUD,1,OPERA_L3_DSWx-HLS_T17RNL_20241003T155516Z_2024...,C2617126679-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-05T03:14:28.583Z,2024-10-03T15:55:16.990Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,"[/home/conda/input_dir/GSHHS_f_L1.dbf, /home/c...","[{'SizeUnit': 'MB', 'Size': 0.2147130966186523...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-HLS_T17RNL_2024...,2024-10-05T02:45:47.159Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-81.04986800575617, 27.07425010377366, -79.93...","POLYGON ((-79.93249 27.07425, -79.93249 28.069..."
4,51.680571,granule,G3257775429-POCLOUD,1,OPERA_L3_DSWx-HLS_T17RML_20241003T155540Z_2024...,C2617126679-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-05T03:11:34.649Z,2024-10-03T15:55:40.877Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,"[/home/conda/input_dir/GSHHS_f_L1.dbf, /home/c...","[{'SizeUnit': 'MB', 'Size': 0.8738307952880859...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-HLS_T17RML_2024...,2024-10-05T02:46:00.860Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-82.06750687970883, 27.07425010377366, -80.95...","POLYGON ((-80.95013 27.07425, -80.95013 28.069..."


In [20]:
### Plot the location of the tiles 
#dswx_gdf_PreFlood.explore(fill=False)
m_HLS = leafmap.Map(center=(c_lat, c_lon), zoom=5)
style = { "color": "blue",        # Line color
        "weight": 2,            # Line thickness
        "fillOpacity": 0.0      # Transparent fill
        }
m_HLS.add_gdf(dswx_gdf_PreEvent, layer_name="DSWX Pre-Event", style=style)
m_HLS

Map(center=[28.411106673880376, -81.4585408260787], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [21]:
### Plot the location of the tiles 
#dswx_gdf_SynEvent.explore(fill=False)
m_S1 = leafmap.Map(center=(c_lat, c_lon), zoom=5)
m_S1.add_gdf(dswx_gdf_SynEvent, layer_name="DSWX Syn-Event", style=style)
m_S1

Map(center=[28.411106673880376, -81.4585408260787], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### Query the OPERA DSWx-S1 dataset for the AOI

In [22]:
try:
    dswx_results_PreEvent_S1V1, dswx_gdf_PreEvent_S1V1 = leafmap.nasa_data_search(
        short_name='OPERA_L3_DSWX-S1_V1',
        cloud_hosted=True,
        bounding_box= AOI,
        temporal=(StartDate_PreEvent, EndDate_PreEvent),
        count=-1,  # use -1 to return all datasets
        return_gdf=True,
    )
    
    dswx_results_SynEvent_S1V1, dswx_gdf_SynEvent_S1V1 = leafmap.nasa_data_search(
        short_name='OPERA_L3_DSWX-S1_V1',
        cloud_hosted=True,
        bounding_box= AOI,
        temporal=(StartDate_SynEvent, EndDate_SynEvent),
        count=-1,  # use -1 to return all datasets
        return_gdf=True,
    )
except Exception as e:
    print(f"❌ Error fetching {'OPERA_L3_DSWX-S1_V1'}")

In [23]:
dswx_results_PreEvent_S1V1[0] #Note this just shows a single MGRS/HLS tile

Collection: {'Version': '1.0', 'ShortName': 'OPERA_L3_DSWX-S1_V1'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -81.04986800575617, 'SouthBoundingCoordinate': 27.07425010377366, 'EastBoundingCoordinate': -79.93249312029117, 'NorthBoundingCoordinate': 28.069479381349602}]}}}
Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2024-10-03T23:29:15.000Z', 'BeginningDateTime': '2024-10-03T23:28:54.000Z'}}
Size(MB): 5.644000053405762
Data: ['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T17RNL_20241003T232853Z_20241004T111807Z_S1A_30_v1.0_B01_WTR.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T17RNL_20241003T232853Z_20241004T111807Z_S1A_30_v1.0_B02_BWTR.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T17RNL_20241003T232853Z_20241004T111807Z_S1A_30_v1.0_B03_CONF.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T17RNL_20241003T232853Z_20241004T111807Z_S1A_30_v1.0_B04_DIAG.tif']

In [24]:
dswx_results_SynEvent_S1V1[0] #Note this just shows a single MGRS/HLS tile

Collection: {'Version': '1.0', 'ShortName': 'OPERA_L3_DSWX-S1_V1'}
Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -82.07663581871259, 'SouthBoundingCoordinate': 27.976885922012322, 'EastBoundingCoordinate': -80.94970544104575, 'NorthBoundingCoordinate': 28.972139864314535}]}}}
Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2024-10-11T11:26:36.000Z', 'BeginningDateTime': '2024-10-11T11:26:13.000Z'}}
Size(MB): 10.657270431518555
Data: ['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T17RMM_20241011T112643Z_20241011T180545Z_S1A_30_v1.0_B01_WTR.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T17RMM_20241011T112643Z_20241011T180545Z_S1A_30_v1.0_B02_BWTR.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T17RMM_20241011T112643Z_20241011T180545Z_S1A_30_v1.0_B03_CONF.tif', 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-S1_V1/OPERA_L3_DSWx-S1_T17RMM_20241011T112643Z_20241011T180545Z_S1A_30_v1.0_B04_DIAG.tif']

In [25]:
dswx_gdf_PreEvent_S1V1.head()

,size,concept-type,concept-id,revision-id,native-id,collection-concept-id,provider-id,format,revision-date,EndingDateTime,...,RelatedUrls,InputGranules,ArchiveAndDistributionInformation,DayNightFlag,Identifiers,ProductionDateTime,URL,Name,bbox,geometry
0,5.644000,granule,G3256808476-POCLOUD,1,OPERA_L3_DSWx-S1_T17RNL_20241003T232853Z_20241...,C2949811996-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-04T11:21:26.001Z,2024-10-03T23:29:15.000Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,[OPERA_L2_RTC-S1_T048-101107-IW3_20241003T2328...,"[{'SizeUnit': 'MB', 'Size': 0.3330898284912109...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-S1_T17RNL_20241...,2024-10-04T10:51:33.520Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-81.04986800575617, 27.07425010377366, -79.93...","POLYGON ((-79.93249 27.07425, -79.93249 28.069..."
1,8.088883,granule,G3256808483-POCLOUD,1,OPERA_L3_DSWx-S1_T17RML_20241003T232853Z_20241...,C2949811996-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-04T11:21:26.366Z,2024-10-03T23:29:17.000Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,[OPERA_L2_RTC-S1_T048-101107-IW3_20241003T2328...,"[{'SizeUnit': 'MB', 'Size': 3.0517578125e-05, ...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-S1_T17RML_20241...,2024-10-04T10:51:33.520Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-82.06750687970883, 27.07425010377366, -80.95...","POLYGON ((-80.95013 27.07425, -80.95013 28.069..."
2,5.040950,granule,G3256808916-POCLOUD,1,OPERA_L3_DSWx-S1_T17RLL_20241003T232853Z_20241...,C2949811996-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-04T11:21:58.043Z,2024-10-03T23:29:20.000Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,[OPERA_L2_RTC-S1_T048-101107-IW3_20241003T2328...,"[{'SizeUnit': 'MB', 'Size': 0.2133398056030273...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-S1_T17RLL_20241...,2024-10-04T10:51:33.520Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-83.08473322918799, 27.063050477966776, -81.9...","POLYGON ((-81.95913 27.06305, -81.95913 28.066..."
3,4.099424,granule,G3256808515-POCLOUD,1,OPERA_L3_DSWx-S1_T17RNM_20241003T232853Z_20241...,C2949811996-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-04T11:21:28.267Z,2024-10-03T23:29:30.000Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,[OPERA_L2_RTC-S1_T048-101107-IW3_20241003T2328...,"[{'SizeUnit': 'MB', 'Size': 2.203166961669922,...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-S1_T17RNM_20241...,2024-10-04T10:51:33.520Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-81.05029455895425, 27.976885922012322, -79.9...","POLYGON ((-79.92336 27.97689, -79.92336 28.972..."
4,10.843494,granule,G3256808472-POCLOUD,1,OPERA_L3_DSWx-S1_T17RMM_20241003T232853Z_20241...,C2949811996-POCLOUD,POCLOUD,application/vnd.nasa.cmr.umm+json,2024-10-04T11:21:25.750Z,2024-10-03T23:29:32.000Z,...,[{'URL': 'https://archive.podaac.earthdata.nas...,[OPERA_L2_RTC-S1_T048-101107-IW3_20241003T2328...,"[{'SizeUnit': 'MB', 'Size': 0.470947265625, 'C...",Unspecified,[{'Identifier': 'OPERA_L3_DSWx-S1_T17RMM_20241...,2024-10-04T10:51:33.520Z,https://cdn.earthdata.nasa.gov/umm/granule/v1.6.6,UMM-G,"(-82.07663581871259, 27.976885922012322, -80.9...","POLYGON ((-80.94971 27.97689, -80.94971 28.972..."


*******************************************************************

## Searching and Visualizing NASA OPERA Data Products Interactively 
* Another option is available through an Earthdata login. You can register for an account at urs.earthdata.nasa.gov. 
* For further information please refer to https://leafmap.org/notebooks/101_nasa_opera/
* Run the Jupyter Notebook until this cell, before using the option below

In [26]:
leafmap.nasa_data_login()

The map is zoomed in on the defined AOI. Select a dataset from the Short Name dropdown list. Click the "Search" button to load the available datasets for the region. The footprints of the datasets will be displayed on the map. Click on a footprint to display the metadata of the dataset.

In [27]:
m = leafmap.Map(center=(c_lat, c_lon), zoom=8, height="700px")
m.add_basemap("Satellite")
m.add("NASA_OPERA")
m

Map(center=[28.411106673880376, -81.4585408260787], controls=(ZoomControl(options=['position', 'zoom_in_text',…

The footprints of the datasets can be accessed as a GeoPandas GeoDataFrame:

In [28]:
m._NASA_DATA_GDF.head()

AttributeError: 'Map' object has no attribute '_NASA_DATA_GDF'